In [1]:
# !pip install datasets
# !pip install rank_bm25
# !pip install jamo
# !pip install customized_konlpy
# !pip install konlpy
# !git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
# !cd Mecab-ko-for-Google-Colab && bash install_mecab-ko_on_colab190912.sh

In [71]:
import json
import zipfile
from datasets import load_from_disk
from collections import OrderedDict
from tqdm.notebook import tqdm
import pandas as pd
from collections import Counter

from konlpy.tag import Mecab
from ckonlpy.tag import Twitter
from pororo import Pororo

In [72]:
train_dataset = load_from_disk('/opt/ml/code/data/train_dataset/train')
print(train_dataset)

Dataset({
    features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
    num_rows: 3952
})


In [3]:
valid_dataset = load_from_disk('/opt/ml/code/data/train_dataset/validation')
print(valid_dataset)

Dataset({
    features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
    num_rows: 240
})


In [4]:
test_dataset = load_from_disk('/opt/ml/code/data/test_dataset/validation')
print(test_dataset)

Dataset({
    features: ['id', 'question'],
    num_rows: 600
})


In [1]:
mecab = Mecab()
twitter = Twitter()

In [50]:
corpus_train = train_dataset['question'] + valid_dataset['question']
corpus_test = test_dataset['question']

In [66]:
compounds_test = []
josa = list(filter(lambda x: len(x) <= 2, twitter.dictionary._pos2words['Josa'])) + ['때', '끝', '위', '상', '하', '앞', '뒤', '후', '뜻', '시', '군', '구', '리', '도']
for corp in tqdm(corpus_test):
    flag = 0
    ngrams = []
    pos = mecab.pos(corp)
    for i in range(len(pos)-3):
        if flag == 1:
            flag += 1
        elif flag == 2:
            flag = 0
            
        if (pos[i][1] == pos[i+1][1] == pos[i+2][1] == pos[i+3][1] == 'NNG') and (
            len(pos[i][0]) == 1 or len(pos[i+1][0]) == 1 or len(pos[i+2][0]) == 1 or len(pos[i+3][0]) == 1) and (
                pos[i][0] not in josa and pos[i+1][0] not in josa and pos[i+2][0] not in josa and pos[i+3][0] not in josa):
            string = pos[i][0] + pos[i+1][0] + pos[i+2][0] + pos[i+3][0]
            ngrams.append(string)
            flag = 1

        elif (pos[i][1] == pos[i+1][1] == pos[i+2][1] == 'NNG') and (
            len(pos[i][0]) == 1 or len(pos[i+1][0]) == 1 or len(pos[i+2][0]) == 1) and (
                pos[i][0] not in josa and pos[i+1][0] not in josa and pos[i+2][0] not in josa):
            string = pos[i][0] + pos[i+1][0] + pos[i+2][0]
            ngrams.append(string)
            flag = 1

        elif not flag and (pos[i][1] == pos[i+1][1] == 'NNG') and (
            len(pos[i][0]) == 1 or len(pos[i+1][0]) == 1) and (
                pos[i][0] not in josa and pos[i+1][0] not in josa):
            string = pos[i][0] + pos[i+1][0]
            ngrams.append(string)
                
    compounds_test += list(set(ngrams))
compounds_test = Counter(compounds_test)

In [67]:
import re
def iskorean(word):
    reg = re.compile(r'[가-힣]')
    return word if reg.match(word) else False

comp_freq = list(filter(lambda x: iskorean(x[0]) and x[1] > 0, compounds_test.items()))
comp_freq = list(zip(*comp_freq))[0]
print(len(comp_freq), '/', len(compounds_test))

71 / 71


In [68]:
import pickle
with open('/opt/ml/code/data/compounds_test.pickle', 'wb') as f:
    pickle.dump(comp_freq, f, pickle.HIGHEST_PROTOCOL)

In [69]:
comp_freq

('짓시티계획안',
 '루돌',
 '전주도달산',
 '철툇자루',
 '개편안',
 '전상대학교',
 '디피에트로',
 '공로장',
 '축구대표팀',
 '감독직',
 '쌀가게배달원',
 '대표팀훈련',
 '국가대표팀훈련',
 '전진실위원회이후',
 '창아래',
 '귀상태',
 '왕설',
 '꿈속',
 '저고리밑',
 '진화압',
 '튀폰',
 '극설',
 '차원상자속입자',
 '속입자문제',
 '상자속입자문제',
 '강전투',
 '키프리',
 '국부제거술',
 '시나이산',
 '소속팀',
 '헌법소장직',
 '청백당',
 '선열차',
 '소더스트롬',
 '코미트',
 '당선전',
 '합동조사반',
 '씨종가',
 '당나라왕',
 '섬정부',
 '지시내리기전',
 '로마제국황제관',
 '제국황제관',
 '종교개혁전',
 '경상좌도수군절도사영',
 '좌도수군절도사영',
 '차사건',
 '테로스',
 '맥더프',
 '돈후안호세',
 '베마의스승',
 '홉베마의스승',
 '세비키',
 '시장직',
 '좌운검',
 '골트이후',
 '급수대',
 '어버이새',
 '트푀',
 '쇼자마',
 '방패막',
 '탑사이',
 '리파리섬',
 '이전병',
 '스쿨조사관',
 '서머힐스쿨조사관',
 '중북부지방',
 '전면퇴칸',
 '암목조관음보살좌상',
 '악기옆',
 '곤충명')